installing required packages

In [ ]:
! pip install -r requirements.txt

In [21]:
import requests
import json
from dotenv import load_dotenv
load_dotenv("local.env", override=True)
import os

# Azure OpenAI Related
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")
aoai_api_base_url = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_embedding_deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# Azure AI Search Related
search_api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
search_api_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_api_version = os.getenv("AZURE_SEARCH_API_VERSION")

generating embedding from Azure OpenAI Rest API

In [25]:
# Define the URL of the API endpoint
embedding_url = f"{aoai_api_base_url}openai/deployments/{aoai_embedding_deployment_name}/embeddings?api-version={aoai_api_version}"
# Define the headers, including the API key
embedding_headers = {
    'Content-Type': 'application/json',
    'api-key': aoai_api_key
}

# Define the JSON body of the request
embedding_body = {
    "input": "What is the deductable amount in the health plan?"
}

# Convert the payload to a JSON string
str_body = json.dumps(embedding_body)

# Make the POST request with the headers and JSON body
response = requests.post(url=embedding_url, headers=embedding_headers, data=str_body)

embedding = None
# Check the response
if response.status_code == 200:
    print('Request successful!')
    json_response = response.json()
    embedding = json_response["data"][0]["embedding"]
else:
    print('Request failed!')
    print(response.text)

print(embedding)

Request successful!
[0.024826122, 0.0069515803, 0.026662137, -0.019783732, -0.04071165, 0.029695554, -0.028711025, -0.035788998, -0.027247533, 0.009419558, 0.024107682, 0.039753728, -0.017109536, 0.007862936, 0.0074571497, 0.012758977, 0.026741965, -0.020369127, 0.021952359, 0.00020112601, -0.028498152, 0.012067145, -0.025690911, 0.028524762, 0.0056610475, 0.009851952, 0.023216283, -0.010756656, -0.0023681938, -0.0119607095, 0.04092452, 0.011129181, -0.020635217, -0.013590505, -0.0022118664, 0.032702364, -0.0061133993, 6.7821566e-05, 0.021167396, -0.015326737, 0.025132125, 0.014661515, 0.013011762, -0.0040578605, -0.0074172365, 0.012685803, 0.020302605, -0.0018476569, -0.023176368, -0.00076874834, -0.004709779, 0.007929458, -0.035629343, 0.0006340407, -0.012898674, -0.008069155, 0.0030134602, -0.009532645, 0.017335711, -0.006386141, 0.012572714, -0.017588496, -0.015180388, 0.011302139, -0.025797348, 0.0001750368, 0.012991805, 0.011548271, -0.026010219, 0.0066954694, 0.026329527, 0.0370

Calling Azure AI Search Service to do vector search

for this sample, we are calling a search index that has vectorized content of "Breaking Bad - 01 Pilot" screenplay

In [26]:
# Variables
search_index_name = "contoso-northwind"
search_index_vector_field = "vector"

# The URL for the request
search_url = f"{search_api_endpoint}/indexes/{search_index_name}/docs/search?api-version={search_api_version}"

# The headers for the request
search_headers = {
    "Content-Type": "application/json",
    "api-key": search_api_key
}

# The body of the request
search_body = {
    "count": True,
    "select": "title, chunk", # other fields to return may go here
    "vectorQueries": [
        {
            "kind": "vector",
            "vector": embedding,
            "exhaustive": True,
            "fields":search_index_vector_field,
            "k": 5
        }
    ]
}

# Make the POST request
response = requests.post(url=search_url, headers=search_headers, data=json.dumps(search_body))

# Check the response
if response.status_code == 200:
    print('Request successful!')
    json_response = response.json()
    json_str = json.dumps(json_response, indent=2)
    # Print the response
    print(json_str)
else:
    print('Request failed!')
    print(response.text)


Request successful!
{
  "@odata.context": "https://jhl-ai-search.search.windows.net/indexes('contoso-northwind')/$metadata#docs(*)",
  "@odata.count": 5,
  "value": [
    {
      "@search.score": 0.8689805,
      "chunk": "the office staff that you are a Northwind \n\nHealth Plus member. This will ensure that the staff bills your insurance correctly and that \n\nyou are charged the correct amount for your copay.  \n\nIf you have any questions about your copay amount, you can contact Northwind Health's \n\ncustomer service team. They will be able to provide more detailed information about your \n\ncoverage and copay amount.  \n\nIt is important to remember that the split copay only applies to office visits. Other services, \n\nsuch as lab tests, X-rays, and imaging tests, are subject to different copay amounts, which \n\nare listed on your insurance card.  \n\nIt is also important to remember that the split copay only applies to in-network providers. If \n\nyou visit an out-of-network p

In [28]:
# Variables
search_index_vector_profile = "contoso-northwind"
search_index_vector_field = "vector"

# The URL for the request
search_url = f"{search_api_endpoint}/indexes/{search_index_name}/docs/search?api-version={search_api_version}"

# The headers for the request
search_headers = {
    "Content-Type": "application/json",
    "api-key": search_api_key
}

# The body of the request
search_body = {
    "count": True,
    "select": "title, chunk", # other fields to return may go here
    "vectorQueries": [
        {
            "kind": "text",
            "text": "What is the deductable amount in the health plan?",
            "exhaustive": True,
            "fields": search_index_vector_field,
            "k": 5
        }
    ]
}

# Make the POST request
response = requests.post(url=search_url, headers=search_headers, data=json.dumps(search_body))

# Check the response
if response.status_code == 200:
    print('Request successful!')
    json_response = response.json()
    json_str = json.dumps(json_response, indent=2)
    # Print the response
    print(json_str)
else:
    print('Request failed!')
    print(response.text)


Request successful!
{
  "@odata.context": "https://jhl-ai-search.search.windows.net/indexes('contoso-northwind')/$metadata#docs(*)",
  "@odata.count": 5,
  "value": [
    {
      "@search.score": 0.8689805,
      "chunk": "the office staff that you are a Northwind \n\nHealth Plus member. This will ensure that the staff bills your insurance correctly and that \n\nyou are charged the correct amount for your copay.  \n\nIf you have any questions about your copay amount, you can contact Northwind Health's \n\ncustomer service team. They will be able to provide more detailed information about your \n\ncoverage and copay amount.  \n\nIt is important to remember that the split copay only applies to office visits. Other services, \n\nsuch as lab tests, X-rays, and imaging tests, are subject to different copay amounts, which \n\nare listed on your insurance card.  \n\nIt is also important to remember that the split copay only applies to in-network providers. If \n\nyou visit an out-of-network p

: 